In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
import shutil
from torchvision import datasets, transforms

from model import ResNet50, ResNet38, ResNet26



In [2]:

## 데이터 셋은 CIFAR10으로
num_classes = 10
batch_size = 32
num_workers = 4

## 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True, transform=transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

Files already downloaded and verified


In [3]:
stem = False
model = ResNet26(num_classes=num_classes, stem=stem)
model = model.cuda()

In [4]:
start_epoch = 1
best_acc = 0.0

lr = 0.01
momentum =0.9
weight_decay = 0.0001


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


In [5]:
epochs = 100
print_interval = 20

## 30에폭 마다 러닝레이트 줄임
def adjust_learning_rate(optimizer, epoch):
    decayed_lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = decayed_lr

for epoch in range(1,epochs+1):
    model.train()
    train_acc = 0.0
    step = 0
    for data, target in train_loader:
        adjust_learning_rate(optimizer, epoch)
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        ## onnx 저장


        y_pred = output.data.max(1)[1]
        
        acc = float(y_pred.eq(target.data).sum()) / len(data) * 100.
        train_acc += acc
        step += 1
        if step % print_interval == 0:
            print("Epoch ",epoch," ",len(train_loader)," / ",step,"batch Loss: ",loss.item(),"  ACC: ",acc,"%")

    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            prediction = output.data.max(1)[1]
            correct += prediction.eq(target.data).sum()

    acc = 100. * float(correct) / len(test_loader.dataset)
    print('Test acc: {0:.2f} \n'.format(acc))

Epoch  1   1563  /  20 batch Loss:  3.2431983947753906   ACC:  21.875 %
Epoch  1   1563  /  40 batch Loss:  2.280831813812256   ACC:  15.625 %
Epoch  1   1563  /  60 batch Loss:  2.4326906204223633   ACC:  21.875 %
Epoch  1   1563  /  80 batch Loss:  2.344860553741455   ACC:  25.0 %
Epoch  1   1563  /  100 batch Loss:  2.339205741882324   ACC:  12.5 %
Epoch  1   1563  /  120 batch Loss:  2.096317768096924   ACC:  21.875 %
Epoch  1   1563  /  140 batch Loss:  2.139641523361206   ACC:  9.375 %
Epoch  1   1563  /  160 batch Loss:  2.032835006713867   ACC:  18.75 %
Epoch  1   1563  /  180 batch Loss:  2.1409754753112793   ACC:  18.75 %
Epoch  1   1563  /  200 batch Loss:  2.08933687210083   ACC:  31.25 %
Epoch  1   1563  /  220 batch Loss:  2.170435905456543   ACC:  15.625 %
Epoch  1   1563  /  240 batch Loss:  1.9093222618103027   ACC:  21.875 %
Epoch  1   1563  /  260 batch Loss:  2.041768789291382   ACC:  21.875 %
Epoch  1   1563  /  280 batch Loss:  1.6941570043563843   ACC:  31.25 %
E

KeyboardInterrupt: 

In [ ]:
## onnx 저장
## operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK 꼭 해줘야함
## unfold 연산을 
# https://github.com/pytorch/pytorch/issues/14395#issuecomment-444697403


# import torch.onnx
# torch.onnx.export(model.cpu(),data.cpu(),'./stand_alone_self.onnx',export_params=False,opset_version=12,
#                   operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)